In [1088]:
#!pip install rake_nltk
#!pip install --upgrade pip

In [1089]:
import nltk
import pandas as pd
import re
import unicodedata
import numpy as np

from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer #tokenizes a collection of words extracted from a text doc
from ast import literal_eval #This evaluates whether an expresion is a Python datatype or not


In [1090]:
cosmetics_path = "./data/cosmetics.csv"
raw_df = pd.read_csv(cosmetics_path)
raw_df.head()

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolatum, Glycerin, Isohexadecane, Microcrystalline Wax, Lanolin Alcohol, Citrus Aurantifolia (Lime) Extract, Sesamum Indicum (Sesame) Seed Oil, Eucalyptus Globulus (Eucalyptus) Leaf Oil, Sesamum Indicum (Sesame) Seed Powder, Medicago Sativa (Alfalfa) Seed Powder, Helianthus Annuus (Sunflower) Seedcake, Prunus Amygdalus Dulcis (Sweet Almond) Seed Meal, Sodium Gluconate, Copper Gluconate, Calcium Gluconate, Magnesium Gluconate, Zinc Gluconate, Magnesium Sulfate, Paraffin, Tocopheryl Succinate, Niacin, Water, Beta-Carotene, Decyl Oleate, Aluminum Distearate, Octyldodecanol, Citric Acid, Cyanocobalamin, Magnesium Stearate, Panthenol, Limonene, Geraniol, Linalool, Hydroxycitronellal, Citronellol, Benzyl Salicylate, Citral, Sodium Benzoate, Alcohol Denat., Fragrance.",1,1,1,1,1
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butylene Glycol, Pentylene Glycol, Water, Sodium Benzoate, Methylparaben, Sorbic Acid.",1,1,1,1,1
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Cetearyl Alcohol, Cetearyl Olivate, Sorbitan Olivate, Sclerocarya Birrea Seed Oil, Bacillus/Soybean/ Folic Acid Ferment Extract, Nymphaea Alba Root Extract, sh-Oligopeptide-1, sh-Oligopeptide-2, sh-Polypeptide-1, sh-Polypeptide-9, sh-Polypeptide-11, Copper Palmitoyl Heptapeptide-14, Heptapeptide-15 Palmitate, Palmitoyl Tetrapeptide-7, Palmitoyl Tripeptide-1, Alanine, Arginine, Glycine, Histidine, Isoleucine, Phenylalanine, Proline, Serine, Threonine, Valine, Acetyl Glutamine, Coconut Alkanes , Coco-Caprylate/Caprate, Sodium Hyaluronate, Aspartic Acid, Linoleic Acid, Linolenic Acid, Lecithin, Butylene Glycol, Polyvinyl Alcohol, Sodium Lactate, Sodium PCA, PCA, Sorbitan Isostearate, Carbomer, Polysorbate 20, Polysorbate 60, Lactic Acid/Glycolic Acid Copolymer, Hydroxyethyl Acrylate/Sodium Acryloyldimethyl Taurate Copolymer, Xanthan Gum, Isomalt, 1,2-Hexanediol, Caprylyl Glycol, Chlorphenesin, Phenoxyethanol, Tocopherol, Sodium Benzoate, Phenylpropanol, Glyceryl Caprylate, Symphytum Officinale Callus Culture Extract.",1,1,1,1,0
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, Petrolatum, Glyceryl Distearate, Phenyl Trimethicone, Butylene Glycol, Hydrogenated Vegetable Oil, Cholesterol, Butyrospermum Parkii (Shea Butter), Steareth-10, Dimethicone, Glyceryl Stearate Se, Polysilicone-11, Sesamum Indicum (Sesame) Seed Oil, Medicago Sativa (Alfalfa) Seed Powder, Helianthus Annuus (Sunflower) Seedcake, Prunus Amygdalus Dulcis (Sweet Almond) Seed Meal, Eucalyptus Globulus (Eucalyptus) Leaf Oil, Sodium Gluconate, Copper Gluconate, Calcium Gluconate, Magnesium Gluconate, Zinc Gluconate, Tocopheryl Succinate, Niacin, Sesamum Indicum (Sesame) Seed Powder, Water, Citrus Aurantifolia (Lime) Peel Extract, Laminaria Digitata Extract, Crithmum Maritimum Extract, Salicornia Herbacea Extract, Plankton Extract, Chlorella Vulgaris Extract, Glycine Soja (Soybean) Seed Extract, Glycerin, Caffeine, Sea Salt/Maris Sal/Sel Marin, Micrococcus Lysate, Diethylhexyl Succinate, Adenosine Phosphate, Creatine, Hydrolyzed Algin, Isocetyl Stearoyl Stearate, Cetyl Alcohol, Sucrose, Acetyl Hexapeptide-8, Glucose Oxidase, Polyacrylamide, Acetyl Carnitine Hcl, Glucose, Caprylic/Capric Triglyceride, C13-14 Isoparaffin, Tocopheryl Acetate, Tetrahexyldecyl Ascorbate, Sodium Pca, Glycosaminoglycans, Urea, Distearyldimonium Chloride, Dipalmitoyl Hydroxyproline, Sodium Hyaluronate, Laureth-7, Lecithin, Trehalose, Polyquaternium-51, Lactoperoxidase, Hydroxypropyl Cyclodextrin, Cyanocobalamin, Pentylene Glycol, Fragrance, Disodium Edta, Bht, Citronellol, Hydroxycitronellal, Geraniol, Linalool, Limonene, Potassium Sorbate, Phenoxyethanol",1,1,1,1,1
4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ C

In [1091]:
raw_df.describe()

,Price,Rank,Combination,Dry,Normal,Oily,Sensitive
count,1472.000000,1472.000000,1472.00000,1472.000000,1472.000000,1472.000000,1472.000000
mean,55.584239,4.153261,0.65625,0.614130,0.652174,0.607337,0.513587
std,45.014429,0.633918,0.47512,0.486965,0.476442,0.488509,0.499985
min,3.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,4.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,42.500000,4.300000,1.00000,1.000000,1.000000,1.000000,1.000000
75%,68.000000,4.500000,1.00000,1.000000,1.000000,1.000000,1.000000
max,370.000000,5.000000,1.00000,1.000000,1.000000,1.000000,1.000000


In [1092]:
raw_df.isnull().sum()

Label          0
Brand          0
Name           0
Price          0
Rank           0
Ingredients    0
Combination    0
Dry            0
Normal         0
Oily           0
Sensitive      0
dtype: int64

In [1093]:
raw_df.columns

Index(['Label', 'Brand', 'Name', 'Price', 'Rank', 'Ingredients', 'Combination',
       'Dry', 'Normal', 'Oily', 'Sensitive'],
      dtype='object')

In [1094]:
#pasamos todas las columnas te tipo string a minusculas, si no son strings las deja tal cual
with_types = with_types.map(lambda content: content.lower() if type(content) == str else content)

In [1095]:
with_types["Ingredients"]

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [1096]:
def clean_ingredients(text):
    #quita los enters y los cambia por espacios en blanco
    text =text.replace("\n", " ")
    #si contiene "ingredientes:" divide el string en antes y despues de ingredients: gogemos lo que viene despes unicamente y con el strip removemos todos los espacios en blanco al principio y al final del string
    if "ingredients:" in text:
        text = text.split("ingredients:")[1].strip()
    #elimina las partes del texto que esten entre dos puntos : y ppunto siempre que no contenga una coma . esto esta hecho despues de los ingredientes a posta porque hubiera borrdo todo lo anterior y nunca hubiera entrado en ese if
    text = re.sub(r':((?!.*?,).*?)\.', '', text)
    text =text.replace(" -", ", ")
    text = text.replace(" =;", ",")
    if text.startswith("-"):
        text =text.replace("-", "")
    text =text.replace("(", "")
    text =text.replace(")", "")
    text =text.replace("*", "")
    list_of_text = text.split(", ")

    text = ", ".join([item.replace(" ", "") for item in list_of_text])
    if text.startswith("visit") or text.startswith("#name?"):
        return "No info"
    return text

with_types["Clean Ingredients"] = with_types["Ingredients"].apply(clean_ingredients)
with_types["Clean Ingredients"].head(50)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [1097]:
with_types["Clean Ingredients"].head(50)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [1098]:
def clean_names(text):
    normalized = unicodedata.normalize("NFD", text)
    without_accents = "".join(character for character in normalized if unicodedata.category(character) != "Mn")
    cleaned = re.sub(r"[^a-zA-Z0-9\s]", "", without_accents)
    return cleaned.lower()

In [1099]:
with_types["Clean Name"] = with_types["Name"].apply(clean_names)
with_types[["Clean Name", "Name"]]

,Clean Name,Name
0,creme de la mer,crème de la mer
1,facial treatment essence,facial treatment essence
2,protini polypeptide cream,protini™ polypeptide cream
3,the moisturizing soft cream,the moisturizing soft cream
4,your skin but better cc cream with spf 50,your skin but better™ cc+™ cream with spf 50+
...,...,...
1467,yoghurt nourishing fluid veil face sunscreen broad spectrum spf 30,yoghurt nourishing fluid veil face sunscreen broad spectrum spf 30
1468,daily deflector waterlight broad spectrum spf 50 pa antiaging sunscreen,daily deflector™ waterlight broad spectrum spf 50+ pa+++ anti-aging sunscreen
1469,self tan dry oil spf 50,self tan dry oil spf 50
1470,pro light self tan bronzing mist,pro light self tan bronzing mist


In [1100]:
with_types.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Label              1472 non-null   object 
 1   Brand              1472 non-null   object 
 2   Name               1472 non-null   object 
 3   Price              1472 non-null   int64  
 4   Rank               1472 non-null   float64
 5   Ingredients        1472 non-null   object 
 6   Combination        1472 non-null   int64  
 7   Dry                1472 non-null   int64  
 8   Normal             1472 non-null   int64  
 9   Oily               1472 non-null   int64  
 10  Sensitive          1472 non-null   int64  
 11  Skin Types         1472 non-null   object 
 12  Clean Ingredients  1472 non-null   object 
 13  Clean Name         1472 non-null   object 
dtypes: float64(1), int64(6), object(7)
memory usage: 161.1+ KB


In [1101]:
dataframe_cosine = with_types[["Label", "Brand", "Clean Name", "Clean Ingredients", "Skin Types", "Rank"]].copy()
dataframe_cosine.head()

,Label,Brand,Clean Name,Clean Ingredients,Skin Types,Rank
0,moisturizer,la mer,creme de la mer,"algaeseaweedextract, mineraloil, petrolatum, glycerin, isohexadecane, microcrystallinewax, lanolinalcohol, citrusaurantifolialimeextract, sesamumindicumsesameseedoil, eucalyptusglobuluseucalyptusleafoil, sesamumindicumsesameseedpowder, medicagosativaalfalfaseedpowder, helianthusannuussunflowerseedcake, prunusamygdalusdulcissweetalmondseedmeal, sodiumgluconate, coppergluconate, calciumgluconate, magnesiumgluconate, zincgluconate, magnesiumsulfate, paraffin, tocopherylsuccinate, niacin, water, beta-carotene, decyloleate, aluminumdistearate, octyldodecanol, citricacid, cyanocobalamin, magnesiumstearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzylsalicylate, citral, sodiumbenzoate, alcoholdenat., fragrance.",combination dry normal oily sensitive alltypes,4.1
1,moisturizer,sk-ii,facial treatment essence,"galactomycesfermentfiltratepitera, butyleneglycol, pentyleneglycol, water, sodiumbenzoate, methylparaben, sorbicacid.",combination dry normal oily sensitive alltypes,4.1
2,moisturizer,drunk elephant,protini polypeptide cream,"water, dicaprylylcarbonate, glycerin, cetearylalcohol, cetearylolivate, sorbitanolivate, sclerocaryabirreaseedoil, bacillus/soybean/folicacidfermentextract, nymphaeaalbarootextract, sh-oligopeptide-1, sh-oligopeptide-2, sh-polypeptide-1, sh-polypeptide-9, sh-polypeptide-11, copperpalmitoylheptapeptide-14, heptapeptide-15palmitate, palmitoyltetrapeptide-7, palmitoyltripeptide-1, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetylglutamine, coconutalkanes, coco-caprylate/caprate, sodiumhyaluronate, asparticacid, linoleicacid, linolenicacid, lecithin, butyleneglycol, polyvinylalcohol, sodiumlactate, sodiumpca, pca, sorbitanisostearate, carbomer, polysorbate20, polysorbate60, lacticacid/glycolicacidcopolymer, hydroxyethylacrylate/sodiumacryloyldimethyltauratecopolymer, xanthangum, isomalt, 1,2-hexanediol, caprylylglycol, chlorphenesin, phenoxyethanol, tocopherol, sodiumbenzoate, phenylpropanol, glycerylcaprylate, symphytumofficinalecalluscultureextract.",combination dry normal oily,4.4
3,moisturizer,la mer,the moisturizing soft cream,"algaeseaweedextract, cyclopentasiloxane, petrolatum, glyceryldistearate, phenyltrimethicone, butyleneglycol, hydrogenatedvegetableoil, cholesterol, butyrospermumparkiisheabutter, steareth-10, dimethicone, glycerylstearatese, polysilicone-11, sesamumindicumsesameseedoil, medicagosativaalfalfaseedpowder, helianthusannuussunflowerseedcake, prunusamygdalusdulcissweetalmondseedmeal, eucalyptusglobuluseucalyptusleafoil, sodiumgluconate, coppergluconate, calciumgluconate, magnesiumgluconate, zincgluconate, tocopherylsuccinate, niacin, sesamumindicumsesameseedpowder, water, citrusaurantifolialimepeelextract, laminariadigitataextract, crithmummaritimumextract, salicorniaherbaceaextract, planktonextract, chlorellavulgarisextract, glycinesojasoybeanseedextract, glycerin, caffeine, seasalt/marissal/selmarin, micrococcuslysate, diethylhexylsuccinate, adenosinephosphate, creatine, hydrolyzedalgin, isocetylstearoylstearate, cetylalcohol, sucrose, acetylhexapeptide-8, glucoseoxidase, polyacrylamide, acetylcarnitinehcl, glucose, caprylic/caprictriglyceride, c13-14isoparaffin, tocopherylacetate, tetrahexyldecylascorbate, sodiumpca, glycosaminoglycans, urea, distearyldimoniumchloride, dipalmitoylhydroxyproline, sodiumhyaluronate, laureth-7, lecithin, trehalose, polyquaternium-51, lactoperoxidase, hydroxypropylcyclodextrin, cyanocobalamin, pentyleneglycol, fragrance, disodiumedta, bht, citronellol, hydroxycitronellal, geraniol, linalool, limonene, potassiumsorbate, phenoxyethanol",combination dry normal oily sensitive alltypes,3.8
4,moisturizer,it cosmetics,your skin but better cc cream with spf 50,"water, snailsecretionfiltrate, phenyltrimethicone, dimethicone, butyleneglycol, butyleneglycoldicaprylate/dicaprate, or

In [1102]:
query = "no info"
dataframe_cosine[ dataframe_cosine["Clean Ingredients"].str.contains(query)]

,Label,Brand,Clean Name,Clean Ingredients,Skin Types,Rank


# Empezamos con las keywords desde aqui

In [1103]:
dataframe_cosine["keywords"] = ""

In [1104]:
# nltk.download('stopwords')

In [1105]:
for index, row in dataframe_cosine.iterrows():
    ingredients = row["Clean Ingredients"]

    r = Rake()

    r.extract_keywords_from_text(ingredients)
    keyword_dict_scores = r.get_word_degrees()

    dataframe_cosine.at[index, "keywords"] = list(keyword_dict_scores.keys())

In [1106]:
dataframe_cosine[["Clean Ingredients", "keywords"]]

,Clean Ingredients,keywords
0,"algaeseaweedextract, mineraloil, petrolatum, glycerin, isohexadecane, microcrystallinewax, lanolinalcohol, citrusaurantifolialimeextract, sesamumindicumsesameseedoil, eucalyptusglobuluseucalyptusleafoil, sesamumindicumsesameseedpowder, medicagosativaalfalfaseedpowder, helianthusannuussunflowerseedcake, prunusamygdalusdulcissweetalmondseedmeal, sodiumgluconate, coppergluconate, calciumgluconate, magnesiumgluconate, zincgluconate, magnesiumsulfate, paraffin, tocopherylsuccinate, niacin, water, beta-carotene, decyloleate, aluminumdistearate, octyldodecanol, citricacid, cyanocobalamin, magnesiumstearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzylsalicylate, citral, sodiumbenzoate, alcoholdenat., fragrance.","[algaeseaweedextract, mineraloil, petrolatum, glycerin, isohexadecane, microcrystallinewax, lanolinalcohol, citrusaurantifolialimeextract, sesamumindicumsesameseedoil, eucalyptusglobuluseucalyptusleafoil, sesamumindicumsesameseedpowder, medicagosativaalfalfaseedpowder, helianthusannuussunflowerseedcake, prunusamygdalusdulcissweetalmondseedmeal, sodiumgluconate, coppergluconate, calciumgluconate, magnesiumgluconate, zincgluconate, magnesiumsulfate, paraffin, tocopherylsuccinate, niacin, water, beta, carotene, decyloleate, aluminumdistearate, octyldodecanol, citricacid, cyanocobalamin, magnesiumstearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzylsalicylate, citral, sodiumbenzoate, alcoholdenat, .,, fragrance]"
1,"galactomycesfermentfiltratepitera, butyleneglycol, pentyleneglycol, water, sodiumbenzoate, methylparaben, sorbicacid.","[galactomycesfermentfiltratepitera, butyleneglycol, pentyleneglycol, water, sodiumbenzoate, methylparaben, sorbicacid]"
2,"water, dicaprylylcarbonate, glycerin, cetearylalcohol, cetearylolivate, sorbitanolivate, sclerocaryabirreaseedoil, bacillus/soybean/folicacidfermentextract, nymphaeaalbarootextract, sh-oligopeptide-1, sh-oligopeptide-2, sh-polypeptide-1, sh-polypeptide-9, sh-polypeptide-11, copperpalmitoylheptapeptide-14, heptapeptide-15palmitate, palmitoyltetrapeptide-7, palmitoyltripeptide-1, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetylglutamine, coconutalkanes, coco-caprylate/caprate, sodiumhyaluronate, asparticacid, linoleicacid, linolenicacid, lecithin, butyleneglycol, polyvinylalcohol, sodiumlactate, sodiumpca, pca, sorbitanisostearate, carbomer, polysorbate20, polysorbate60, lacticacid/glycolicacidcopolymer, hydroxyethylacrylate/sodiumacryloyldimethyltauratecopolymer, xanthangum, isomalt, 1,2-hexanediol, caprylylglycol, chlorphenesin, phenoxyethanol, tocopherol, sodiumbenzoate, phenylpropanol, glycerylcaprylate, symphytumofficinalecalluscultureextract.","[water, dicaprylylcarbonate, glycerin, cetearylalcohol, cetearylolivate, sorbitanolivate, sclerocaryabirreaseedoil, bacillus, soybean, folicacidfermentextract, nymphaeaalbarootextract, sh, oligopeptide, 1, 2, polypeptide, 9, 11, copperpalmitoylheptapeptide, 14, heptapeptide, 15palmitate, palmitoyltetrapeptide, 7, palmitoyltripeptide, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetylglutamine, coconutalkanes, coco, caprylate, caprate, sodiumhyaluronate, asparticacid, linoleicacid, linolenicacid, lecithin, butyleneglycol, polyvinylalcohol, sodiumlactate, sodiumpca, pca, sorbitanisostearate, carbomer, polysorbate20, polysorbate60, lacticacid, glycolicacidcopolymer, hydroxyethylacrylate, sodiumacryloyldimethyltauratecopolymer, xanthangum, isomalt, hexanediol, caprylylglycol, chlorphenesin, phenoxyethanol, tocopherol, sodiumbenzoate, phenylpropanol, glycerylcaprylate, symphytumofficinalecalluscultureextract]"
3,"algaeseaweedextract, cyclopentasiloxane, petrolatum, glyceryldistearate, phenyltrimethicone, butyleneglycol, hydrogenatedvegetableoil, cholesterol, butyrospermumparkiisheabutter, steareth-10, dimethicone, glycery

In [1107]:
dataframe_cosine[dataframe_cosine["Clean Name"].duplicated()]

,Label,Brand,Clean Name,Clean Ingredients,Skin Types,Rank,keywords


In [1108]:
dataframe_cosine  = dataframe_cosine.set_index("Clean Name")

In [1109]:
dataframe_cosine

,Label,Brand,Clean Ingredients,Skin Types,Rank,keywords
Clean Name,,,,,,
creme de la mer,moisturizer,la mer,"algaeseaweedextract, mineraloil, petrolatum, glycerin, isohexadecane, microcrystallinewax, lanolinalcohol, citrusaurantifolialimeextract, sesamumindicumsesameseedoil, eucalyptusglobuluseucalyptusleafoil, sesamumindicumsesameseedpowder, medicagosativaalfalfaseedpowder, helianthusannuussunflowerseedcake, prunusamygdalusdulcissweetalmondseedmeal, sodiumgluconate, coppergluconate, calciumgluconate, magnesiumgluconate, zincgluconate, magnesiumsulfate, paraffin, tocopherylsuccinate, niacin, water, beta-carotene, decyloleate, aluminumdistearate, octyldodecanol, citricacid, cyanocobalamin, magnesiumstearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzylsalicylate, citral, sodiumbenzoate, alcoholdenat., fragrance.",combination dry normal oily sensitive alltypes,4.1,"[algaeseaweedextract, mineraloil, petrolatum, glycerin, isohexadecane, microcrystallinewax, lanolinalcohol, citrusaurantifolialimeextract, sesamumindicumsesameseedoil, eucalyptusglobuluseucalyptusleafoil, sesamumindicumsesameseedpowder, medicagosativaalfalfaseedpowder, helianthusannuussunflowerseedcake, prunusamygdalusdulcissweetalmondseedmeal, sodiumgluconate, coppergluconate, calciumgluconate, magnesiumgluconate, zincgluconate, magnesiumsulfate, paraffin, tocopherylsuccinate, niacin, water, beta, carotene, decyloleate, aluminumdistearate, octyldodecanol, citricacid, cyanocobalamin, magnesiumstearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzylsalicylate, citral, sodiumbenzoate, alcoholdenat, .,, fragrance]"
facial treatment essence,moisturizer,sk-ii,"galactomycesfermentfiltratepitera, butyleneglycol, pentyleneglycol, water, sodiumbenzoate, methylparaben, sorbicacid.",combination dry normal oily sensitive alltypes,4.1,"[galactomycesfermentfiltratepitera, butyleneglycol, pentyleneglycol, water, sodiumbenzoate, methylparaben, sorbicacid]"
protini polypeptide cream,moisturizer,drunk elephant,"water, dicaprylylcarbonate, glycerin, cetearylalcohol, cetearylolivate, sorbitanolivate, sclerocaryabirreaseedoil, bacillus/soybean/folicacidfermentextract, nymphaeaalbarootextract, sh-oligopeptide-1, sh-oligopeptide-2, sh-polypeptide-1, sh-polypeptide-9, sh-polypeptide-11, copperpalmitoylheptapeptide-14, heptapeptide-15palmitate, palmitoyltetrapeptide-7, palmitoyltripeptide-1, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetylglutamine, coconutalkanes, coco-caprylate/caprate, sodiumhyaluronate, asparticacid, linoleicacid, linolenicacid, lecithin, butyleneglycol, polyvinylalcohol, sodiumlactate, sodiumpca, pca, sorbitanisostearate, carbomer, polysorbate20, polysorbate60, lacticacid/glycolicacidcopolymer, hydroxyethylacrylate/sodiumacryloyldimethyltauratecopolymer, xanthangum, isomalt, 1,2-hexanediol, caprylylglycol, chlorphenesin, phenoxyethanol, tocopherol, sodiumbenzoate, phenylpropanol, glycerylcaprylate, symphytumofficinalecalluscultureextract.",combination dry normal oily,4.4,"[water, dicaprylylcarbonate, glycerin, cetearylalcohol, cetearylolivate, sorbitanolivate, sclerocaryabirreaseedoil, bacillus, soybean, folicacidfermentextract, nymphaeaalbarootextract, sh, oligopeptide, 1, 2, polypeptide, 9, 11, copperpalmitoylheptapeptide, 14, heptapeptide, 15palmitate, palmitoyltetrapeptide, 7, palmitoyltripeptide, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetylglutamine, coconutalkanes, coco, caprylate, caprate, sodiumhyaluronate, asparticacid, linoleicacid, linolenicacid, lecithin, butyleneglycol, polyvinylalcohol, sodiumlactate, sodiumpca, pca, sorbitanisostearate, carbomer, polysorbate20, polysorbate60, lacticacid, glycolicacidcopolymer, hydroxyethylacrylate, sodiumacryloyldimethyltauratecopolymer, xanthangum, isomalt, hexanediol, caprylylglycol, chlorphenesin, phenoxyethanol, tocopherol, sodi

In [1110]:
dataframe_cosine.drop("Clean Ingredients", axis=1, inplace=True)

In [1111]:
dataframe_cosine.columns

Index(['Label', 'Brand', 'Skin Types', 'Rank', 'keywords'], dtype='object')

In [1112]:
without_rank = dataframe_cosine[['Label', 'Skin Types', 'keywords']]
columns  = without_rank.columns
dataframe_cosine["bow"] = ""

for index, row in dataframe_cosine.iterrows():
    words = ""
    for column in columns:
        if isinstance(row[column], str):
            words = words + row[column] + " "
        if isinstance(row[column], list):
            words = words + " ".join(row[column])+ " "
        dataframe_cosine.at[index, "bow"] = words

In [1113]:
dataframe_cosine.head(20)

,Label,Brand,Skin Types,Rank,keywords,bow
Clean Name,,,,,,
creme de la mer,moisturizer,la mer,combination dry normal oily sensitive alltypes,4.1,"[algaeseaweedextract, mineraloil, petrolatum, glycerin, isohexadecane, microcrystallinewax, lanolinalcohol, citrusaurantifolialimeextract, sesamumindicumsesameseedoil, eucalyptusglobuluseucalyptusleafoil, sesamumindicumsesameseedpowder, medicagosativaalfalfaseedpowder, helianthusannuussunflowerseedcake, prunusamygdalusdulcissweetalmondseedmeal, sodiumgluconate, coppergluconate, calciumgluconate, magnesiumgluconate, zincgluconate, magnesiumsulfate, paraffin, tocopherylsuccinate, niacin, water, beta, carotene, decyloleate, aluminumdistearate, octyldodecanol, citricacid, cyanocobalamin, magnesiumstearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzylsalicylate, citral, sodiumbenzoate, alcoholdenat, .,, fragrance]","moisturizer combination dry normal oily sensitive alltypes algaeseaweedextract mineraloil petrolatum glycerin isohexadecane microcrystallinewax lanolinalcohol citrusaurantifolialimeextract sesamumindicumsesameseedoil eucalyptusglobuluseucalyptusleafoil sesamumindicumsesameseedpowder medicagosativaalfalfaseedpowder helianthusannuussunflowerseedcake prunusamygdalusdulcissweetalmondseedmeal sodiumgluconate coppergluconate calciumgluconate magnesiumgluconate zincgluconate magnesiumsulfate paraffin tocopherylsuccinate niacin water beta carotene decyloleate aluminumdistearate octyldodecanol citricacid cyanocobalamin magnesiumstearate panthenol limonene geraniol linalool hydroxycitronellal citronellol benzylsalicylate citral sodiumbenzoate alcoholdenat ., fragrance"
facial treatment essence,moisturizer,sk-ii,combination dry normal oily sensitive alltypes,4.1,"[galactomycesfermentfiltratepitera, butyleneglycol, pentyleneglycol, water, sodiumbenzoate, methylparaben, sorbicacid]",moisturizer combination dry normal oily sensitive alltypes galactomycesfermentfiltratepitera butyleneglycol pentyleneglycol water sodiumbenzoate methylparaben sorbicacid
protini polypeptide cream,moisturizer,drunk elephant,combination dry normal oily,4.4,"[water, dicaprylylcarbonate, glycerin, cetearylalcohol, cetearylolivate, sorbitanolivate, sclerocaryabirreaseedoil, bacillus, soybean, folicacidfermentextract, nymphaeaalbarootextract, sh, oligopeptide, 1, 2, polypeptide, 9, 11, copperpalmitoylheptapeptide, 14, heptapeptide, 15palmitate, palmitoyltetrapeptide, 7, palmitoyltripeptide, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetylglutamine, coconutalkanes, coco, caprylate, caprate, sodiumhyaluronate, asparticacid, linoleicacid, linolenicacid, lecithin, butyleneglycol, polyvinylalcohol, sodiumlactate, sodiumpca, pca, sorbitanisostearate, carbomer, polysorbate20, polysorbate60, lacticacid, glycolicacidcopolymer, hydroxyethylacrylate, sodiumacryloyldimethyltauratecopolymer, xanthangum, isomalt, hexanediol, caprylylglycol, chlorphenesin, phenoxyethanol, tocopherol, sodiumbenzoate, phenylpropanol, glycerylcaprylate, symphytumofficinalecalluscultureextract]",moisturizer combination dry normal oily water dicaprylylcarbonate glycerin cetearylalcohol cetearylolivate sorbitanolivate sclerocaryabirreaseedoil bacillus soybean folicacidfermentextract nymphaeaalbarootextract sh oligopeptide 1 2 polypeptide 9 11 copperpalmitoylheptapeptide 14 heptapeptide 15palmitate palmitoyltetrapeptide 7 palmitoyltripeptide alanine arginine glycine histidine isoleucine phenylalanine proline serine threonine valine acetylglutamine coconutalkanes coco caprylate caprate sodiumhyaluronate asparticacid linoleicacid linolenicacid lecithin butyleneglycol polyvinylalcohol sodiumlactate sodiumpca pca sorbitanisostearate carbomer polysorbate20 polysorbate60 lacticacid glycolicacidcopolymer hydroxyethylacrylate sodiumacryloyldimethyltauratecopolymer xanthangum isomalt hexanediol caprylylglycol chlorphenesin phenoxyethanol tocopherol sodiumbenzoate phenyl

In [1114]:
dataframe_cosine.shape

(1472, 6)

In [1115]:
len(dataframe_cosine["bow"].iloc(0)[1].split(" "))

15

In [1116]:
count = CountVectorizer()
count_matrix = count.fit_transform(dataframe_cosine["bow"])

In [1117]:
count_matrix.shape

(1472, 5557)

In [1118]:
dataframe_cosine.index

Index(['creme de la mer', 'facial treatment essence',
       'protini polypeptide cream', 'the moisturizing soft cream',
       'your skin but better cc cream with spf 50', 'the water cream',
       'lala retro whipped cream', 'virgin marula luxury facial oil',
       'ultra facial cream',
       'little miss miracle limitededition creme de la mer',
       ...
       'summer duo', 'passionflower  argan dry oil broad spectrum spf 50',
       'antiaging armour super smart skinperfecting beauty fluid spf 50',
       'naked skin bronzing beauty balm broad spectrum spf 20',
       'daily deflector moisturizer broad spectrum spf 50',
       'yoghurt nourishing fluid veil face sunscreen broad spectrum spf 30',
       'daily deflector waterlight broad spectrum spf 50 pa antiaging sunscreen',
       'self tan dry oil spf 50', 'pro light self tan bronzing mist',
       'dermaprotect daily defense broad spectrum spf 50'],
      dtype='object', name='Clean Name', length=1472)

In [1119]:
list(count.vocabulary_.items())

[('moisturizer', 3330),
 ('combination', 1599),
 ('dry', 1942),
 ('normal', 3455),
 ('oily', 3531),
 ('sensitive', 4595),
 ('alltypes', 469),
 ('algaeseaweedextract', 458),
 ('mineraloil', 3321),
 ('petrolatum', 3834),
 ('glycerin', 2288),
 ('isohexadecane', 2794),
 ('microcrystallinewax', 3313),
 ('lanolinalcohol', 2909),
 ('citrusaurantifolialimeextract', 1438),
 ('sesamumindicumsesameseedoil', 4607),
 ('eucalyptusglobuluseucalyptusleafoil', 2069),
 ('sesamumindicumsesameseedpowder', 4608),
 ('medicagosativaalfalfaseedpowder', 3205),
 ('helianthusannuussunflowerseedcake', 2448),
 ('prunusamygdalusdulcissweetalmondseedmeal', 4168),
 ('sodiumgluconate', 4749),
 ('coppergluconate', 1630),
 ('calciumgluconate', 1006),
 ('magnesiumgluconate', 3123),
 ('zincgluconate', 5524),
 ('magnesiumsulfate', 3130),
 ('paraffin', 3743),
 ('tocopherylsuccinate', 5203),
 ('niacin', 3438),
 ('water', 5439),
 ('beta', 839),
 ('carotene', 1150),
 ('decyloleate', 1772),
 ('aluminumdistearate', 516),
 ('octy

In [1120]:
# cosine_similarity?

In [1121]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.34016803, 0.13522468, ..., 0.24494897, 0.232379  ,
        0.28284271],
       [0.34016803, 1.        , 0.25555063, ..., 0.4114756 , 0.34156503,
        0.53452248],
       [0.13522468, 0.25555063, 1.        , ..., 0.18401748, 0.17457431,
        0.15936381],
       ...,
       [0.24494897, 0.4114756 , 0.18401748, ..., 1.        , 0.42163702,
        0.51320024],
       [0.232379  , 0.34156503, 0.17457431, ..., 0.42163702, 1.        ,
        0.4868645 ],
       [0.28284271, 0.53452248, 0.15936381, ..., 0.51320024, 0.4868645 ,
        1.        ]])

In [1122]:
indices = pd.Series(dataframe_cosine.index)

indices

0                                                               creme de la mer
1                                                      facial treatment essence
2                                                     protini polypeptide cream
3                                                   the moisturizing soft cream
4                                     your skin but better cc cream with spf 50
                                         ...                                   
1467         yoghurt nourishing fluid veil face sunscreen broad spectrum spf 30
1468    daily deflector waterlight broad spectrum spf 50 pa antiaging sunscreen
1469                                                    self tan dry oil spf 50
1470                                           pro light self tan bronzing mist
1471                           dermaprotect daily defense broad spectrum spf 50
Name: Clean Name, Length: 1472, dtype: object

In [1123]:
indices[indices == "lala retro whipped cream"].index[0]

np.int64(6)

In [1124]:
pd.Series(cosine_sim[1]).sort_values(ascending=False)[1:5+1]

1      1.000000
116    0.897085
288    0.661438
169    0.661438
240    0.661438
dtype: float64

In [1125]:
dataframe_cosine.iloc[116]

Label                                                                                                                                                                            moisturizer
Brand                                                                                                                                                                                  sk-ii
Skin Types                                                                                                                                    combination dry normal oily sensitive alltypes
Rank                                                                                                                                                                                     0.0
keywords                                                        [galactomycesfermentfiltrate, butyleneglycol, pentyleneglycol, water, sodiumbenzoate, methylparaben, sorbicacid, pitera, ™.]
bow           moisturizer combination dry normal oily s

In [1126]:
dataframe_cosine.iloc[1]

Label                                                                                                                                                                        moisturizer
Brand                                                                                                                                                                              sk-ii
Skin Types                                                                                                                                combination dry normal oily sensitive alltypes
Rank                                                                                                                                                                                 4.1
keywords                                                          [galactomycesfermentfiltratepitera, butyleneglycol, pentyleneglycol, water, sodiumbenzoate, methylparaben, sorbicacid]
bow           moisturizer combination dry normal oily sensitive alltypes ga

In [1127]:
def recommend_cosmetic(name, n = 5):
    recommended_list = []
    idx = indices[indices == name].index[0]

    min_recommend_value = 0.2

    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending=False)
    score_series = score_series[score_series >= min_recommend_value]
    print(score_series.values)
    top_n_indexes = list(score_series.iloc[1:n+1].index)
    print(top_n_indexes)

    for i in top_n_indexes:

        recommended_list.append(list(dataframe_cosine.index)[i])

    return dataframe_cosine.loc[recommended_list]

In [1131]:
name = "ultra repair cream intense hydration"
recommend_cosmetic(name)

[1.         0.59375    0.55       ... 0.20044593 0.20044593 0.20044593]
[157, 1357, 892, 141, 187]


,Label,Brand,Skin Types,Rank,keywords,bow
Clean Name,,,,,,
ultra repair face moisturizer,moisturizer,first aid beauty,combination dry normal oily sensitive alltypes,4.2,"[water, glycerylstearatese, glycerin, caprylic, caprictriglyceride, cetylalcohol, collodialoatmeal, dimethicone, squalane, urea, allantoin, ceramidenp, butyrospermumparkiisheabutter, limnanthesalbameadowfoamseedoil, perseagratissimaavocadooil, chrysanthemumpartheniumfeverfewextract, camelliasinensisleafextract, glycyrrhizaglabralicoricerootextract, caprylylglycol, phenoxyethanol, carbomer, sodiumhydroxide, leuconostoc, radishrootfermentfiltrate, edta]",moisturizer combination dry normal oily sensitive alltypes water glycerylstearatese glycerin caprylic caprictriglyceride cetylalcohol collodialoatmeal dimethicone squalane urea allantoin ceramidenp butyrospermumparkiisheabutter limnanthesalbameadowfoamseedoil perseagratissimaavocadooil chrysanthemumpartheniumfeverfewextract camelliasinensisleafextract glycyrrhizaglabralicoricerootextract caprylylglycol phenoxyethanol carbomer sodiumhydroxide leuconostoc radishrootfermentfiltrate edta
ultra repair pure mineral sunscreen moisturizer broad spectrum spf 40,sun protect,first aid beauty,combination dry normal oily sensitive alltypes,3.6,"[water, butyloctylsalicylate, cetearylalcohol, caprylic, caprictriglyceride, steareth, 2, dimethicone, squalane, glycerin, butyleneglycol, alumina, ceteareth, 20, colloidaloatmeal, urea, allantoin, butyrospermumparkiisheabutter, copaiferaofficinalisbalsamcopaibaresin, limnanthesalbameadowfoamseedoil, perseagratissimaavocadooil, chrysanthemumpartheniumfeverfewextract, camelliasinensisleafextract, glycyrrhizaglabralicoricerootextract, ceramidenp, tocopherylacetate, triethoxycaprylylsilane, caprylylglycol, isopropyltitaniumtriisostearate, c13, 14isoparaffin, c12, 15alkylbenzoate, polyhydroxystearicacid, laureth, 7, methicone, leuconostoc, radishrootfermentfiltrate, xanthangum, polyacrylamide, phenoxyethanol, disodiumedta, edta]",sun protect combination dry normal oily sensitive alltypes water butyloctylsalicylate cetearylalcohol caprylic caprictriglyceride steareth 2 dimethicone squalane glycerin butyleneglycol alumina ceteareth 20 colloidaloatmeal urea allantoin butyrospermumparkiisheabutter copaiferaofficinalisbalsamcopaibaresin limnanthesalbameadowfoamseedoil perseagratissimaavocadooil chrysanthemumpartheniumfeverfewextract camelliasinensisleafextract glycyrrhizaglabralicoricerootextract ceramidenp tocopherylacetate triethoxycaprylylsilane caprylylglycol isopropyltitaniumtriisostearate c13 14isoparaffin c12 15alkylbenzoate polyhydroxystearicacid laureth 7 methicone leuconostoc radishrootfermentfiltrate xanthangum polyacrylamide phenoxyethanol disodiumedta edta
ultra repair instant oatmeal mask,face mask,first aid beauty,combination dry normal oily sensitive alltypes,4.4,"[water, colloidaloatmeal, glycerylstearatecitrate, cetylalcohol, glycerylstearatese, caprylic, capric, myristic, stearictriglyceride, butyleneglycoldicaprylate, dicaprate, butyrospermumparkiibutter, octyldodecanol, glycerin, lactobacillusferment, polysorbate20, cetearylalcohol, decylglucoside, theobromacacaoseedbutter, polysorbate60, prunusarmeniacakerneloil, xanthangum, tetrasodiumedta, dimethicone, bisabolol, camelliasinensisleafextract, chrysanthemumpartheniumextract, glycyrrhizaglabrarootextract, hippophaerhamnoidesextract, vanillaplanifoliafruitextract, vanillin, avenasativakernelextract, zingiber, officinalerootextract, phenoxyethanol, potassiumsorbate]",face mask combination dry normal oily sensitive alltypes water colloidaloatmeal glycerylstearatecitrate cetylalcohol glycerylstearatese caprylic capric myristic stearictriglyceride butyleneglycoldicaprylate dicaprate butyrospermumparkiibutter octyldodecanol glycerin lactobacillusferment polysorbate20 cetearylalcohol decylglucoside theobromacacaoseedbutter polysorbate60 prunusarmeniacakerneloil xanthangum tetrasodiumedta dimethicone bisabolol camelliasinensisleaf

In [1129]:
dataframe_cosine.loc[name]

Label                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    cleanser
Brand                                                                                                                                                                                                                                                                                                 